![](images/softmax for mnist.png)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
import tensorflow as tf

# Clear the graph because we might run this
# cell in Jupyter many times, and we don't
# want multiple copies of the graph.
tf.python.framework.ops.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, 784], name="inputs")
expected_outputs = tf.placeholder(tf.float32, [None, 10], name="weights")
weights = tf.Variable(tf.zeros([784, 10], tf.float32), name="weights")
biases = tf.Variable(tf.zeros([10]), tf.float32, name="biases")

with tf.name_scope("linear_softmax"):
    outputs = tf.nn.softmax(tf.matmul(inputs, weights) + biases)
    
with tf.name_scope("cross_entropy"):
    cross_entropy = -tf.reduce_sum(expected_outputs*tf.log(outputs))

with tf.name_scope("train_step"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_step = optimizer.minimize(cross_entropy)

initialize = tf.initialize_all_variables()

with tf.name_scope("accuracy"):
    labels = tf.argmax(outputs, 1)
    expected_labels = tf.argmax(expected_outputs, 1)
    correct_prediction = tf.equal(labels, expected_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as session:
    session.run(initialize)
    for step in xrange(1000):
        batch_inputs, batch_outputs = mnist.train.next_batch(100)
        session.run(train_step, {
                inputs: batch_inputs,
                expected_outputs: batch_outputs
            })

    print session.run(accuracy, {
                inputs: mnist.test.images,
                expected_outputs: mnist.test.labels
            })

## Add summaries

In [14]:
import tensorflow as tf
from time import strftime

# Clear the graph because we might run this
# cell in Jupyter many times, and we don't
# want multiple copies of the graph.
tf.python.framework.ops.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, 784], name="inputs")
expected_outputs = tf.placeholder(tf.float32, [None, 10], name="weights")
weights = tf.Variable(tf.zeros([784, 10], tf.float32), name="weights")
biases = tf.Variable(tf.zeros([10]), tf.float32, name="biases")

with tf.name_scope("linear_softmax"):
    outputs = tf.nn.softmax(tf.matmul(inputs, weights) + biases)
    
with tf.name_scope("cross_entropy"):
    cross_entropies = -tf.reduce_sum(expected_outputs*tf.log(outputs), 1)
    mean_cross_entropy = tf.reduce_mean(cross_entropies, 0)

with tf.name_scope("train_step"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    train_step = optimizer.minimize(mean_cross_entropy)
    

cross_entropy_summary = tf.scalar_summary("cross_entropy", mean_cross_entropy)
initialize = tf.initialize_all_variables()


with tf.name_scope("accuracy"):
    labels = tf.argmax(outputs, 1)
    expected_labels = tf.argmax(expected_outputs, 1)
    correct_prediction = tf.equal(labels, expected_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as session:
    time = strftime("%Y-%m-%d_%H:%M:%S"),
    train_writer = tf.train.SummaryWriter("../logs/softmax_mnist_train_%s" % time, graph_def=session.graph_def)
    validation_writer = tf.train.SummaryWriter("../logs/softmax_mnist_validation_%s" % time)
    
    session.run(initialize)
    for step in xrange(1000):
        batch_inputs, batch_outputs = mnist.train.next_batch(100)
        session.run(train_step, {
                inputs: batch_inputs,
                expected_outputs: batch_outputs
            })

        if step % 10 == 0:
            train_cost = session.run(cross_entropy_summary, {
                    inputs: mnist.train.images,
                    expected_outputs: mnist.train.labels
                })
            train_writer.add_summary(train_cost, step)

            validation_cost = session.run(cross_entropy_summary, {
                    inputs: mnist.validation.images,
                    expected_outputs: mnist.validation.labels
                })
            validation_writer.add_summary(validation_cost, step)
            
            
    print session.run(accuracy, {
                inputs: mnist.test.images,
                expected_outputs: mnist.test.labels
            })

    train_writer.close()
    validation_writer.close()


0.9105


Now we see on TensorBoard something like this:

![](images/softmax for mnist cost graph.png)